In [20]:
# importing for .py
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify
import datetime as dt 
import numpy as np 
import pandas as pd 

In [33]:
#connecting to the engine
engine =create_engine("sqlite:///Resources/hawaii.sqlite", echo=False)

In [34]:
# Looking at the Base Database 
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement', 'station']

In [35]:
#Base tables of Station & Measurement
Station = Base.classes.station
Measurement = Base.classes.measurement

In [36]:
# creating the session enguine
session = Session(engine)

In [37]:
# Using the Flask Setup
app = Flask(__name__)

In [38]:
# Setting the Flask Routes 
@app.route("/")
def main():
# List all routes that are available
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end>"
    )

In [39]:
@app.route("/api/v1.0/precipitation")
def precipitation():
# Convert the query results to a dictionary using date as the key and prcp as the value.
# Return the JSON representation of your dictionary.
    """Return the JSON representation of your dictionary."""
    print("Received Precipitation API request.")
    
    #Query of the Dates
    final_date_query = session.query(func.max(func.strftime("%Y-%m-%d", Measurement.date))).all()
    max_date_string = final_date_query[0][0]
    max_date = dt.datetime.strptime(max_date_string, "%Y-%m-%d")

    begin_date = max_date - dt.timedelta(365)

    climate_precipitation_data = session.query(func.strftime("%Y-%m-%d", Measurement.date), Measurement.prcp).\
        filter(func.strftime("%Y-%m-%d", Measurement.date) >= begin_date).all()
    
    results_dict = {}
    for result in climate_precipitation_data:
        results_dict[result[0]] = result[1]
    
    return jsonify(results_dict)


In [40]:
#Route to stations
@app.route("/api/v1.0/stations")
def stations():

# Return a JSON list of stations from the dataset.
    
    print("Received the station API request.")

    stations = session.query(Station).all()
    
    # Create a dictionary from the row data and append to a list of stations_list
    station_list = []
    for station in stations:
        station_dict = {}
        station_dict["id"] = station.id
        station_dict["station"] = station.station
        station_dict["name"] = station.name
        station_dict["latitude"] = station.latitude
        station_dict["longitude"] = station.longitude
        station_dict["elevation"] = station.elevation
        stations_list.append(station_dict)
    
    return jsonify(stations_list)


In [43]:
#Route to tobs (Temperature Observations)
@app.route("/api/v1.0/tobs")
def tobs():

    print ("Recieved the tobs API requests.")
    #Query the dates and temperature observations of the most active station for the last year of data.
    final_date_query = session.query(func.max(func.strftime("%Y-%m-%d", Measurement.date))).all()
    max_date_string = final_date_query[0][0]
    max_date = dt.datetime.strptime(max_date_string, "%Y-%m-%d")

    begin_date = max_date - dt.timedelta(365)

    results = session.query(Measurement).\
        filter(func.strftime("%Y-%m-%d", Measurement.date) >= begin_date).all()
    
    # Create a dictionary from the row data and append to a list of tobs_list
    tobs_list = []
    for result in results:
        tobs_dict = {}
        tobs_dict["date"] = result.date
        tobs_dict["station"] = result.station
        tobs_dict["tobs"] = result.tobs
        tobs_list.append(tobs_dict)
    
    return jsonify(tobs_list)


In [45]:
# Route to <start>
@app.route("/api/v1.0/<start>")
def start(start):

    print("Received start date API request.")

    final_date_query = session.query(func.max(func.strftime("%Y-%m-%d", Measurement.date))).all()
    max_date = final_date_query[0][0]

    #add temperatures
    temps = calc_temps(start, max_date)

    #create list
    return_list = []
    date_dict = {'start_date': start, 'end_date': max_date}
    return_list.append(date_dict)
    return_list.append({'Observation': 'TMIN', 'Temperature': temps[0][1]})
    return_list.append({'Observation': 'TAVG', 'Temperature': temps[0][1]})
    return_list.append({'Observation': 'TMAX', 'Temperature': temps[0][0]})

    return jsonify(return_list)


AssertionError: View function mapping is overwriting an existing endpoint function: start

In [46]:
@app.route("/api/v1.0/<start>/<end>")
def start_end(start, end):
    """Return a JSON list of the minimum temperature, the average temperature, and the max temperature for a given start or start-end range."""

    print("Received start date and end date api request.")

    #add temperatures
    temps = calc_temps(start, end)

    #create list
    return_list = []
    date_dict = {'start_date': start, 'end_date': end}
    return_list.append(date_dict)
    return_list.append({'Observation': 'TMIN', 'Temperature': temps[0][1]})
    return_list.append({'Observation': 'TAVG', 'Temperature': temps[0][1]})
    return_list.append({'Observation': 'TMAX', 'Temperature': temps[0][0]})

    return jsonify(return_list)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1